<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Acquiring-the-Data" data-toc-modified-id="Acquiring-the-Data-1">Acquiring the Data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2">Preprocessing</a></span></li><li><span><a href="#Collaborative-Filtering" data-toc-modified-id="Collaborative-Filtering-3">Collaborative Filtering</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Add-movieId-to-input-user" data-toc-modified-id="Add-movieId-to-input-user-3.0.1">Add movieId to input user</a></span></li><li><span><a href="#Collect-the-users-who-has-seen-the-same-movies" data-toc-modified-id="Collect-the-users-who-has-seen-the-same-movies-3.0.2">Collect the users who has seen the same movies</a></span></li><li><span><a href="#Similarity-of-users-to-input-user" data-toc-modified-id="Similarity-of-users-to-input-user-3.0.3">Similarity of users to input user</a></span></li><li><span><a href="#The-top-x-similar-users-to-input-user" data-toc-modified-id="The-top-x-similar-users-to-input-user-3.0.4">The top x similar users to input user</a></span></li><li><span><a href="#Rating-of-all-movies-by-selected-users" data-toc-modified-id="Rating-of-all-movies-by-selected-users-3.0.5">Rating of all movies by selected users</a></span></li></ul></li><li><span><a href="#Advantages-and-Disadvantages-of-Collaborative-Filtering" data-toc-modified-id="Advantages-and-Disadvantages-of-Collaborative-Filtering-3.1">Advantages and Disadvantages of Collaborative Filtering</a></span></li></ul></li></ul></div>

# COLLABORATIVE FILTERING

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

## Acquiring the Data

To acquire and extract the data, simply run the following Bash scripts:  
Dataset acquired from [GroupLens](http://grouplens.org/datasets/movielens/). Lets download the dataset. To download the data, we will use **`!wget`** to download it from IBM Object Storage.  
__Did you know?__ When it comes to Machine Learning, you will likely be working with large datasets. As a business, where can you host your data? IBM is offering a unique opportunity for businesses, with 10 Tb of IBM Cloud Object Storage: [Sign up now for free](http://cocl.us/ML0101EN-IBM-Offer-CC)

```bash
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
!unzip -o -j ./data/moviedataset.zip ./data/
```

Now you're ready to start working with the data!

## Preprocessing

First, let's get all of the imports out of the way:

In [183]:
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("precision", 3)
%matplotlib inline

Now let's read each file into their Dataframes:

In [184]:
PATH = './Data/moviedataset/'

In [185]:
# Storing the movie information into a pandas dataframe
movies_df = pd.read_csv(PATH+'movies.csv')

In [186]:
# Storing the user information into a pandas dataframe
ratings_df = pd.read_csv(PATH+'ratings.csv')

Let's also take a peek at how each of them are organized:

In [187]:
movies_df.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html#pandas.Series.str.extract) function that Pandas has.

Let's remove the year from the __title__ column by using pandas' replace function and store in a new __year__ column.

In [188]:
# Using regular expressions to find a year stored between parentheses
# We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))', expand=False)

In [189]:
# Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)', expand=False)

In [190]:
# Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')

<ipython-input-190-0c57e901ca5e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [191]:
# Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())

Let's look at the result!

In [192]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also drop the genres column since we won't need it for this particular recommendation system.

In [193]:
# Dropping the genres column
movies_df = movies_df.drop('genres', 1)

Here's the final movies dataframe:

In [194]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


Next, let's look at the ratings dataframe.

In [195]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.

In [196]:
# Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)

Here's how the final ratings Dataframe looks like:

In [197]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


## Collaborative Filtering

The technique we're going to take a look at is called __Collaborative Filtering__, which is also known as __User-User Filtering__. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the __Pearson Correlation Function__.


The process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score


Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [198]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


#### Add movieId to input user

With the input complete, let's extract the input movies's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [199]:
# Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputId

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
293,296,Pulp Fiction,1994
1246,1274,Akira,1988
1885,1968,"Breakfast Club, The",1985


In [200]:
# Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,movieId,title,year,rating
0,1,Toy Story,1995,3.5
1,2,Jumanji,1995,2.0
2,296,Pulp Fiction,1994,5.0
3,1274,Akira,1988,4.5
4,1968,"Breakfast Club, The",1985,5.0


In [201]:
# Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('year', 1)

In [202]:
# Final input dataframe
# If a movie you added in above isn't here, then it might not be in the original 
# dataframe or it might spelled differently, please check capitalisation.
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


#### Collect the users who has seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [203]:
# Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


In [204]:
userSubset.shape

(196623, 3)

We now group up the rows by user ID.

In [205]:
# Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

lets look at two of the users, e.g. one with **userID=4** and another with **userID=1130**

In [206]:
userSubsetGroup.get_group(4)

,userId,movieId,rating
19,4,296,4.0


In [207]:
userSubsetGroup.get_group(1130)

,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5
104443,1130,1968,4.5


In [208]:
[print(row) for (i, row) in zip(range(5), userSubsetGroup)]

(4,     userId  movieId  rating
19       4      296     4.0)
(12,      userId  movieId  rating
441      12     1968     3.0)
(13,      userId  movieId  rating
479      13        2     2.0
531      13     1274     5.0)
(14,      userId  movieId  rating
681      14      296     2.0)
(15,      userId  movieId  rating
749      15        1     4.0
776      15      296     3.0
911      15     1968     3.0)


[None, None, None, None, None]

Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [209]:
# Sorting it so users with most movies in common with input user have higher priority
# userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: x[1].shape[0], reverse=True)

Now lets look at the first user

In [210]:
userSubsetGroup[0:3]

[(75,
        userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0),
 (106,
        userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5),
 (686,
         userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the __Pearson Correlation Coefficient__. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

**Why Pearson Correlation?**

>Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .
![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")
The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 
In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

In [211]:
userSubsetGroup = userSubsetGroup[0:100]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient


In [212]:
# Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pc_dict = {}

# For every user group in our subset
for name, group in userSubsetGroup:
    
    # Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    
    # Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())].rating.values
    
    # Let's also put the current user group reviews in a list format
    tempGroupList = group.rating.values
    
    pc_dict[name] = np.corrcoef(temp_df, tempGroupList)[1,0]
    

In [213]:
corr_df = pd.DataFrame.from_dict(pc_dict, orient='index')
corr_df.columns = ['similarityIndex']
corr_df.tail()

,similarityIndex
17854,0.537
17897,0.877
17944,0.271
18301,0.298
18509,0.132


In [214]:
corr_df['userId'] = pearsonDF.index
# pc_df.index rangerange(len(pearsonDF))
corr_df.reset_index(inplace=True, drop=True)
corr_df.tail()

,similarityIndex,userId
95,0.537,17854
96,0.877,17897
97,0.271,17944
98,0.298,18301
99,0.132,18509


#### The top x similar users to input user

Now let's get the top 50 users that are most similar to the input.

In [215]:
topUsers=corr_df.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
64,0.962,12325
34,0.962,6207
55,0.962,10707
67,0.961,13053
4,0.943,1040


Now, let's start recommending movies to the input user.

#### Rating of all movies by selected users
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our __corr_df__ from the ratings dataframe and then store their correlation in a new column called "similarityIndex". This is achieved below by merging of these two tables.

In [216]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.962,12325,1,3.5
1,0.962,12325,2,1.5
2,0.962,12325,3,3.0
3,0.962,12325,5,0.5
4,0.962,12325,6,2.5


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [217]:
# Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

,similarityIndex,userId,movieId,rating,weightedRating
0,0.962,12325,1,3.5,3.366
1,0.962,12325,2,1.5,1.443
2,0.962,12325,3,3.0,2.885
3,0.962,12325,5,0.5,0.481
4,0.962,12325,6,2.5,2.404


In [218]:
topUsersRating.groupby('movieId').sum()

,similarityIndex,userId,rating,weightedRating
movieId,,,,
1,38.376,495942,184.0,140.801
2,38.376,495942,127.5,96.657
3,10.254,151444,35.0,27.254
4,0.929,12120,3.0,2.788
5,11.723,142707,36.0,27.152
...,...,...,...,...
148630,0.716,16456,3.0,2.148
148652,0.577,815,2.0,1.153
149354,0.877,2791,3.0,2.631


In [219]:
# Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,38.376,140.801
2,38.376,96.657
3,10.254,27.254
4,0.929,2.788
5,11.723,27.152


In [220]:
# Creates an empty dataframe
recommendation_df = pd.DataFrame()

# Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.669,1
2,2.519,2
3,2.658,3
4,3.000,4
5,2.316,5


Now let's sort it and see the top 20 movies that the algorithm recommended!

In [221]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
26801,5.0,26801
3539,5.0,3539
1902,5.0,1902
6660,5.0,6660
6668,5.0,6668
121,5.0,121
3851,5.0,3851
90531,5.0,90531
3776,5.0,3776


In [222]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

,movieId,title,year
119,121,"Boys of St. Vincent, The",1992
1819,1902,Dream for an Insomniac,1996
3449,3539,"Filth and the Fury, The",2000
3685,3775,Make Mine Music,1946
3686,3776,Melody Time,1948
3759,3851,I'm the One That I Want,2000
6551,6660,"Red Shoes, The",1948
6559,6668,"Road Home, The (Wo de fu qin mu qin)",1999
9064,26801,Dragon Inn (Sun lung moon hak chan),1992
18106,90531,Shame,2011


### Advantages and Disadvantages of Collaborative Filtering

##### Advantages
* Takes other user's ratings into consideration
* Doesn't need to study or extract information from the recommended item
* Adapts to the user's interests which might change over time

##### Disadvantages
* Approximation function can be slow
* There might be a low of amount of users to approximate
* Privacy issues when trying to learn the user's preferences